### <center>**Analyse des Clusters**</center>

#### **1. Calcule des Moyennes par Cluster :**

In [54]:
import pandas as pd

data = pd.read_csv("../data/trained/data.csv")

moyennes = data.groupby('Cluster').mean()

moyennes

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
Cluster,,,,,,,,
0,0.288980,0.571798,0.388545,0.512102,0.552375,0.494910,0.133976,0.457839
1,-0.314109,-0.621519,-0.422332,-0.556633,-0.600408,-0.537946,-0.145626,-0.497651


- **Cluster 0 - Potentiel risque élevé :**

    - ``Glucose`` : Moyenne élevée (1.04), au-dessus de la moyenne globale càd risque plus élevé de diabète.

    - ``Insulin`` : Moyenne légèrement élevée (5.31) ce qui reflète un métabolisme actif de l’insuline.

    - ``BMI`` : Légèrement au-dessus de la moyenne (3.56) càd un poids légèrement supérieur.

    - ``Pregnancies``, ``BloodPressure``, ``SkinThickness``, ``Age``, ``DiabetesPedigreeFunction`` : Légèrement supérieurs à la moyenne globale, contribuant modérément au profil de risque.

    - **Interprétation générale :** Ce cluster regroupe les individus présentant un profil métabolique plus élevé, donc un risque potentiel de diabète plus important.

- **Cluster 1 - Potentiel risque faible :**

    - ``Glucose`` : Moyenne inférieure (-0.62), au-dessous de la moyenne globale càd risque de diabète plus faible.

    - ``Insulin`` : Moyenne plus basse (4.56) càd activité ``insulin``ique moins élevée.

    - ``BMI`` : Légèrement plus faible (3.45) càd poids inférieur à la moyenne globale.

    - ``Pregnancies``, ``BloodPressure``, ``SkinThickness``, ``Age``, ``DiabetesPedigreeFunction`` : Légèrement inférieurs ou proches de la moyenne globale, contribuant faiblement au risque.

    - **Interprétation générale :** Ce cluster regroupe les individus présentant un profil métabolique moins prononcé, donc un risque potentiel de diabète plus faible.

<br>

#### **2. Identification du Cluster à Risque selon les Seuils :**

Il y a un point très important à comprendre : Une partie de nos données est standardisées et une autre est transformée en log, donc les seuils classiques (**``Glucose > 126``**, **``BMI > 30``**, **``DPF > 0.5``**) ne s’appliqueront pas directement. Il faut faire une transformation inverse pour comparer avec les seuils critiques réels.

In [60]:
import joblib
import numpy as np
from sklearn.preprocessing import StandardScaler

scaler = joblib.load("../utils/scaler.pkl")
kmeans = joblib.load("../models/kmeans_model.pkl")

original_centers = scaler.inverse_transform(kmeans.cluster_centers_)

cols = data.columns.to_list()
cols.remove("Cluster")

df_centers = pd.DataFrame(original_centers, columns=cols)



log_cols = data.columns.to_list()
log_cols.remove("Glucose")
log_cols.remove("Cluster")

for col in log_cols :
    df_centers[col] = np.expm1(df_centers[col])


for i , row in df_centers.iterrows() :
    if row['Glucose'] > 120 and row['BMI'] > 30 and row['DiabetesPedigreeFunction'] > 0.45 :
        print(f"Le Cluster {i} représente --Haut Risque du Diabète--")
    else :
        print(f"Le Cluster {i} représente --Bas Risque du Diabète--")

df_centers.head()

Le Cluster 0 représente --Haut Risque du Diabète--
Le Cluster 1 représente --Bas Risque du Diabète--


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
0,3.631747,139.135000,76.339724,32.665689,175.625347,35.256517,0.479783,36.522882
1,1.912350,102.816848,66.201792,22.491335,87.103061,28.356541,0.399938,26.819277
